---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [91]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [169]:
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    def get_list_of_university_towns():
        towns = pd.read_csv('university_towns.txt', sep='\n',header=None)
        list1 = []
        list2 = []
        for item in towns.iloc[:,0]:
            if item[-6:] == '[edit]':
                temp = item[:-6]
            list1.append(temp)
            list2.append(item.split('(')[0])
        towns['State'] = list1
        towns['RegionName'] = list2
        towns['temp'] = towns.iloc[:,0]
        towns = towns[['State', 'RegionName', 'temp']]
        towns = towns[towns['RegionName'] != towns['temp']]
        towns = towns.reset_index()
        towns = towns[['State','RegionName']].set_index(['State','RegionName'])
        return towns

    get_list_of_university_towns()

Empty DataFrame
Columns: []
Index: [(Alabama, Auburn ), (Alabama, Florence ), (Alabama, Jacksonville ), (Alabama, Livingston ), (Alabama, Montevallo ), (Alabama, Troy ), (Alabama, Tuscaloosa ), (Alabama, Tuskegee ), (Alaska, Fairbanks ), (Arizona, Flagstaff ), (Arizona, Tempe ), (Arizona, Tucson ), (Arkansas, Arkadelphia ), (Arkansas, Conway ), (Arkansas, Fayetteville ), (Arkansas, Jonesboro ), (Arkansas, Magnolia ), (Arkansas, Monticello ), (Arkansas, Russellville ), (Arkansas, Searcy ), (California, Angwin ), (California, Arcata ), (California, Berkeley ), (California, Chico ), (California, Claremont ), (California, Cotati ), (California, Davis ), (California, Irvine ), (California, Isla Vista ), (California, University Park, Los Angeles ), (California, Merced ), (California, Orange ), (California, Palo Alto ), (California, Pomona ), (California, Redlands ), (California, Riverside ), (California, Sacramento ), (California, University District, San Bernardino ), (California, San Diego ), (California, San Luis Obispo ), (California, Santa Barbara ), (California, Santa Cruz ), (California, Turlock ), (California, Westwood, Los Angeles ), (California, Whittier ), (Colorado, Alamosa ), (Colorado, Boulder ), (Colorado, Durango ), (Colorado, Fort Collins ), (Colorado, Golden ), (Colorado, Grand Junction ), (Colorado, Greeley ), (Colorado, Gunnison ), (Colorado, Pueblo, Colorado ), (Connecticut, Fairfield ), (Connecticut, Middletown ), (Connecticut, New Britain ), (Connecticut, New Haven ), (Connecticut, New London ), (Connecticut, Storrs ), (Connecticut, Willimantic ), (Delaware, Dover ), (Delaware, Newark ), (Florida, Ave Maria ), (Florida, Boca Raton ), (Florida, Coral Gables ), (Florida, DeLand ), (Florida, Estero ), (Florida, Gainesville ), (Florida, Orlando ), (Florida, Sarasota ), (Florida, St. Augustine ), (Florida, St. Leo ), (Florida, Tallahassee ), (Florida, Tampa ), (Georgia, Albany ), (Georgia, Athens ), (Georgia, Atlanta ), (Georgia, Carrollton ), (Georgia, Demorest ), (Georgia, Fort Valley ), (Georgia, Kennesaw ), (Georgia, Milledgeville ), (Georgia, Mount Vernon ), (Georgia, Oxford ), (Georgia, Rome ), (Georgia, Savannah ), (Georgia, Statesboro ), (Georgia, Valdosta ), (Georgia, Waleska ), (Georgia, Young Harris ), (Hawaii, Manoa ), (Idaho, Moscow ), (Idaho, Pocatello ), (Idaho, Rexburg ), (Illinois, Carbondale ), (Illinois, Champaign–Urbana ), (Illinois, Charleston ), (Illinois, DeKalb ), (Illinois, Edwardsville ), ...]

[513 rows x 0 columns]

In [190]:
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    def get_recession_start():
        gdp = pd.read_excel('gdplev.xls', skiprows = 5).drop(index=[0,1])
        gdp['Quarter'] = gdp['Unnamed: 4']
        gdp['GDP'] = gdp['GDP in billions of chained 2009 dollars.1']
        gdp = gdp.loc[214:].reset_index()
        gdp  = gdp[['Quarter', 'GDP']]
        list1 = []
        for i in range(len(gdp)-2):
            if (gdp.iloc[i,1] > gdp.iloc[i+1,1] and gdp.iloc[i+1,1] > gdp.iloc[i+2,1]):
                list1.append(gdp.iloc[i,0])
        
        return list1[0]

get_recession_start()

'2008q2'

In [85]:
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    def get_recession_end():
        gdp = pd.read_excel('gdplev.xls', skiprows = 5).drop(index=[0,1])
        gdp['Quarter'] = gdp['Unnamed: 4']
        gdp['GDP'] = gdp['GDP in billions of chained 2009 dollars.1']
        gdp = gdp.loc[247:].reset_index()
        gdp  = gdp[['Quarter', 'GDP']]
        list1 = []
        for i in range(len(gdp)-2):
            if (gdp.iloc[i,1] < gdp.iloc[i+1,1] and gdp.iloc[i+1,1] < gdp.iloc[i+2,1]):
                list1.append(gdp.iloc[i+2,0])

        return list1[0]
    
get_recession_end()

'2009q4'

In [88]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows = 5).drop(index=[0,1])
    gdp['Quarter'] = gdp['Unnamed: 4']
    gdp['GDP'] = gdp['GDP in billions of chained 2009 dollars.1']
    gdp = gdp.loc[247:253].reset_index()
    gdp  = gdp[['Quarter', 'GDP']].set_index('Quarter')
    return gdp['GDP'].idxmin()

get_recession_bottom()

'2009q2'

In [147]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = pd.read_csv('City_Zhvi_AllHomes.csv').replace(states).set_index(['State','RegionName']).sort_values(by=['State','RegionName'])
    temp = house.iloc[:,51:]
    year = 2000
    quart = 1
    for i in range(0,len(temp.columns),3):
        house[str(year)+'q'+str(quart)] = temp.iloc[:,i:i+3].mean(axis=1)
        if quart < 4:
            quart = quart+1
        else:
            quart = 1
            year = year + 1
    house['2016q3'] = house[['2016-07','2016-08']].mean(axis=1)
    house = house.iloc[:,-67:]
    return house

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69200.000000   69466.666667   71133.333333   
          Alabaster        122833.333333  123133.333333  123566.666667   
          Albertville       72900.000000   72633.333333   73600.000000   
          Arab              82066.666667   81266.666667   82333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       79300.000000   78566.666667   80633.333333   
          Bayou La Batre    44300.000000   44533.333333   43866.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54300.000000   54733.333333   55633.333333   
          Boaz              70466.666667   70133.333333   71000.000000   
          Brent             94100.000000   95133.333333   98000.000000   
          Brighton                   NaN            NaN            NaN   
          Brookwood         94100.000000   97600.000000   99966.666667   
          Buhl              93033.333333   96266.666667   96500.000000   
          Calera           110000.000000  109033.333333  107466.666667   
          Center Point      81066.666667   81600.000000   81800.000000   
          Centreville       96400.000000   96933.333333   99500.000000   
          Chalkville        94366.666667   94500.000000   94333.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          166066.666667  167166.666667  166833.333333   
          Chickasaw         53000.000000   54466.666667   55900.000000   
          Chunchula         81466.666667   82000.000000   82866.666667   
          Citronelle        66333.333333   67700.000000   67833.333333   
          Clay             121600.000000  123666.666667  125400.000000   
          Coden             64100.000000   65766.666667   68533.333333   
          Coker            120733.333333  118933.333333  116633.333333   
          Concord           78566.666667   79533.333333   81800.000000   
          Cottondale       101966.666667  104000.000000  105433.333333   
...                                  ...            ...            ...   
Wisconsin Vernon           180400.000000  174866.666667  175533.333333   
          Vienna           180733.333333  181800.000000  184866.666667   
          Vinland          124300.000000  132600.000000  136766.666667   
          Wales                      NaN            NaN            NaN   
          Waterford        119900.000000  119633.333333  121233.333333   
          Waukesha         139600.000000  136833.333333  138066.666667   
          Waunakee         190233.333333  192300.000000  193433.333333   
          Waupun            83966.666667   84300.000000   84600.000000   
          Wausau            69866.666667   70233.333333   70533.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        81533.333333   83666.666667   82000.000000   
          Weston            82133.333333   83600.000000   85333.333333   
          Wheatland        131566.666667  133966.666667  136766.666667   
          Whitelaw          99066.666667  106300.000000  112000.000000   
          Williams Bay     117100.000000  111866.666667  110433.333333   
          Wilson           129366.666667  131166.666667  135466.666667   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        123600.000000  123966.666667  125200.000000   
          Wind Point       146966.666667  141533.333333  144266.666667   
          Wisconsin Dells  102600.000000  104500.000000  105433.333333   
          Wolf River        98066.666667  102266.666667  106266.666667   
          Woodruff                   N

In [186]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    house = convert_housing_data_to_quarters().loc[:,get_recession_start():get_recession_bottom()]
    univ = get_list_of_university_towns()
    comb = pd.merge(house, univ, left_index=True, right_index=True, how='inner')
    comb['uni'] = True
    ans = comb.merge(house, left_index=True, right_index=True, how='outer')
    
    
    
    return comb

run_ttest()

,,2008q2,2008q3,2008q4,2009q1,2009q2,temp
State,RegionName,,,,,,


In [189]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    hdata = convert_housing_data_to_quarters()
    bstart = hdata.columns[hdata.columns.get_loc(start) - 1]
    
    hdata['ratio'] =  hdata[bstart] - hdata[bottom]
    hdata = hdata[[bottom, bstart, 'ratio']]
    hdata = hdata.reset_index()
    
    unitowns_hdata = pd.merge(hdata,unitowns,how='inner',on=['State','RegionName'])
    unitowns_hdata['uni'] = True
    hdata2 = pd.merge(hdata, unitowns_hdata, how='outer', on=['State','RegionName',bottom, bstart, 'ratio'])
    hdata2['uni'] = hdata2['uni'].fillna(False)
    
    ut = hdata2[hdata2['uni'] == True]
    nut = hdata2[hdata2['uni'] == False]
    
    t,p = ttest_ind(ut['ratio'].dropna(), nut['ratio'].dropna())
    
    different = True if p<0.01 else False
    better = "university town" if ut['ratio'].mean() < nut['ratio'].mean() else "non-university town"
    
    return(different, p, better)

run_ttest()

(False, nan, 'non-university town')